### Import necessary libraries, packages, and dependencies 
- Use os to access the operating system
- Use pandas to create DataFrames and perform analysis 
- Use plotly to generate visuals
- Use dotenv to access .env variables 
- Use sqlalchemy to connect to postgres

In [ ]:
import os
import pandas as pd
import plotly.express as px
from dotenv import load_dotenv
from sqlalchemy import create_engine

### Connect to database and retrieve necessary data
- Connect to the 'arbi_db' database 
- Retrieve all data from the 'luffy' table

In [ ]:
# Load environment variables from .env file
load_dotenv()

# Access the variables 
psql_username = os.getenv('PSQL_USERNAME')
psql_password = os.getenv('PSQL_PASSWORD')
psql_host = os.getenv('PSQL_HOST')
psql_port = os.getenv('PSQL_PORT')
db_name = os.getenv('DB_NAME')

# Define the database url
db_url = f"postgresql://{psql_username}:{psql_password}@{psql_host}:{psql_port}/{db_name}" 

# Create the engine object
engine = create_engine(db_url)

# Write ethe SQL query
query = 'SELECT * FROM luffy'

# Read the SQL query into a DataFrame
luffy_df = pd.read_sql(query, engine)

# Show the DataFrame's head
luffy_df.head()


### Data Analysis
- Profit Distribution across Trades
- Hourly Profit Trend
- Spread Percentage Analysis 
- Exchange Performance 
- Time Series Analysis

-Note: Limited data collection - Program ran every 30 seconds over a 4 hour window

In [ ]:
# Convert the current_datetime column to only show the date
luffy_df['current_datetime'] = pd.to_datetime(luffy_df['current_datetime'])
luffy_df['current_datetime'] = luffy_df['current_datetime'].apply(lambda x: x.time())

# Set the index to the current_datetime column
luffy_df.set_index('current_datetime', inplace=True)

luffy_df

##### Profit Distribution across Trades
- Calculate and visualize the distribution of profits across all trades. This provides insights into the consistency and variability of returns. 

In [ ]:
profit_dist = px.histogram(
    luffy_df, 
    x='profit', 
    nbins=20, 
    title='Profit Distribution', 
    labels={'profit': 'Profit', 'count':'Frequency'})

profit_dist.show()

##### 30 Sec Profit Trend
- Examine the 30 second trend of profits to identify patterns and understand the overall performance of the arbitrage over time.

In [ ]:
thirty_sec_profit = luffy_df.groupby('current_datetime')['profit'].sum().reset_index()
thirty_sec_profit_plot = px.line(
    thirty_sec_profit, 
    x='current_datetime',
    y='profit', 
    title='30 Second Profit Trend',
    labels={'current_datetime':'Time', 'profit':'30 Sec Profit'})

thirty_sec_profit_plot.show()

##### Spread Percentage Analysis
- Explore the spread percentage to understand how much profit is generated relative to the spread. Helps to access the efficieny of the arbitrage strategy. 

In [ ]:
spread_percentage_plot = px.scatter(
    luffy_df, 
    x='spread_percentage',
    y='profit',
    title='Spread Percentage vs Profit',
    labels={'spread_percentage':'Spread Percentage', 'profit':'Profit'})

spread_percentage_plot.show()

##### Exchange Performance
- Evaluate the performance of each exchange in terms of the number of trades and profit. This helps in optimizing the selection of exchanges and diversifying risks.

In [ ]:
exchange_performance = luffy_df.groupby('buy_exchange')['profit'].agg(['count', 'sum']).reset_index()
exchange_performance_plot = px.bar(
    exchange_performance,
    x='buy_exchange',
    y=['count', 'sum'],
    barmode='stack',
    title='Exchange Performance',
    labels={'buy_exchange':'Exchange', 'value':'Value'}
)

exchange_performance_plot.show()

##### Cryptocurrency Performance
- Evaluate the performance of each coin in terms of the number of trades and profit. This helps in optimizing the selection of coins and diversifying risks.

In [ ]:
currency_performance = luffy_df.groupby('currency')['profit'].agg(['count', 'sum']).reset_index()
currency_performance_plot = px.bar(
    currency_performance,
    x='currency',
    y=['count', 'sum'],
    barmode='stack',
    title='Currency Performance',
    labels={'currency':'Currency', 'value':'Value'}
)

currency_performance_plot.show()

##### Time Series Analysis
- Analyze the time series data to identify any patterns or trends in the arbitrage opportunities.

In [ ]:
time_series = luffy_df.reset_index()

time_series_plot = px.line(
    time_series,
    x='current_datetime', 
    y='profit', 
    title='Time Series Analysis',
    labels={'current_datetime': 'Time', 'profit':'Profit'}
)
time_series_plot.show()